# 06_benchmark_analysis.ipynb — combine, ROC, CM, McNemar

# Cell 0 — perf env

In [ ]:
# (same perf env)
import os
os.environ.setdefault("OMP_NUM_THREADS", "8")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "8")
os.environ.setdefault("MKL_NUM_THREADS", "8")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "8")
'8'

# Cell 1 — imports & load

In [ ]:
from pathlib import Path
import json, warnings, numpy as np, pandas as pd, matplotlib.pyplot as plt
import pennylane as qml
from pennylane import numpy as pnp
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import (
    roc_auc_score, roc_curve, accuracy_score, precision_recall_fscore_support,
    confusion_matrix, f1_score, average_precision_score, balanced_accuracy_score,
    matthews_corrcoef, classification_report
)

warnings.filterwarnings("ignore")
ROOT = Path("."); PROCESSED = ROOT/"data/processed"; RESULTS = ROOT/"results"
(RESULTS/"metrics").mkdir(parents=True, exist_ok=True)
(RESULTS/"stats").mkdir(parents=True, exist_ok=True)
(RESULTS/"plots").mkdir(parents=True, exist_ok=True)
(RESULTS/"logs").mkdir(parents=True, exist_ok=True)

# --- Run journal: prints + saves for documentation ---
import time
class RunJournal:
    def __init__(self): self.events=[]
    def log(self, step, status, msg, **kw):
        row = {"ts": time.strftime("%Y-%m-%d %H:%M:%S"), "step": step, "status": status, "message": msg}
        row.update(kw); self.events.append(row)
        sym = "✅" if status=="ok" else ("⚠️" if status=="warn" else "❌")
        print(f"{sym} [{step}] {msg}")
    def save(self, base: Path):
        df = pd.DataFrame(self.events)
        md = ["| ts | step | status | message |","|---|---|---|---|"]
        for _,r in df.iterrows(): md.append(f"| {r.ts} | {r.step} | {r.status} | {r.message} |")
        (base.with_suffix(".md")).write_text("\n".join(md), encoding="utf-8")
        (base.with_suffix(".json")).write_text(df.to_json(orient="records", indent=2), encoding="utf-8")

J = RunJournal()

# --- Load encodings/splits (multi-dataset aware fallback) ---
enc_path = PROCESSED/"encodings_all.npz" if (PROCESSED/"encodings_all.npz").exists() else PROCESSED/"encodings.npz"
spl_path = PROCESSED/"splits_pooled.json" if (PROCESSED/"splits_pooled.json").exists() else PROCESSED/"splits.json"
data = np.load(enc_path, allow_pickle=True)
with open(spl_path) as f: SPL = json.load(f)
J.log("load", "ok", f"Loaded {enc_path.name}, {spl_path.name}")

y = data["y"].astype(int)
X_kmer = data["kmer"].astype(np.float32)
X_onehot = data["onehot"].astype(np.float32)
tr = np.array(SPL["train"]); va = np.array(SPL["val"]); te = np.array(SPL["test"])
J.log("splits", "ok", f"train={len(tr)}, val={len(va)}, test={len(te)}, pos_rate={y.mean():.4f}")

# Cell 2 — metric pack

In [ ]:
def _safe_auc(y_true, y_prob):
    try: return roc_auc_score(y_true, y_prob)
    except Exception: return float("nan")

def _safe_ap(y_true, y_prob):
    try: return average_precision_score(y_true, y_prob)
    except Exception: return float("nan")

def _extended_metrics(y_true, y_prob, y_pred, thr):
    cm = confusion_matrix(y_true, y_pred, labels=[0,1])
    tn, fp, fn, tp = cm.ravel()
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="binary", zero_division=0)
    acc = accuracy_score(y_true, y_pred)
    bal = balanced_accuracy_score(y_true, y_pred)
    tnr = tn / (tn + fp) if (tn + fp) else float("nan")  # specificity
    mcc = matthews_corrcoef(y_true, y_pred) if len(np.unique(y_true))==2 else float("nan")
    return dict(acc=acc, prec=prec, rec=rec, f1=f1,
                roc_auc=_safe_auc(y_true, y_prob),
                pr_auc=_safe_ap(y_true, y_prob),
                balanced_acc=bal, specificity=tnr, mcc=mcc, thr=float(thr)), cm

def pack_metrics(y_true, y_prob, split, model, thr=0.5):
    y_pred = (y_prob >= thr).astype(int)
    m, cm = _extended_metrics(y_true, y_prob, y_pred, thr)
    m.update(dict(model=model, split=split))
    return m, y_pred, cm

# Cell 3 — Classical baselines

In [ ]:
# SVM k-mer
svm_kmer = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
    SVC(C=5.0, kernel="rbf", gamma="scale", probability=True, class_weight="balanced", random_state=0)
)
svm_kmer.fit(X_kmer[tr], y[tr])
p_tr = svm_kmer.predict_proba(X_kmer[tr])[:,1]
p_va = svm_kmer.predict_proba(X_kmer[va])[:,1]
p_te = svm_kmer.predict_proba(X_kmer[te])[:,1]

thr_k = max([(t, f1_score(y[va], (p_va>=t).astype(int), zero_division=0))
             for t in np.linspace(0.1,0.9,33)], key=lambda x:x[1])[0]
m1_tr, y1_tr, cm1_tr = pack_metrics(y[tr], p_tr, "train", "SVM_kmer", thr=thr_k)
m1_va, y1_va, cm1_va = pack_metrics(y[va], p_va, "val",   "SVM_kmer", thr=thr_k)
m1_te, y1_te, cm1_te = pack_metrics(y[te], p_te, "test",  "SVM_kmer", thr=thr_k)
J.log("svm_kmer", "ok", f"Val-opt thr={thr_k:.2f}; test F1={m1_te['f1']:.3f}, AUC={m1_te['roc_auc']:.3f}, PR-AUC={m1_te['pr_auc']:.3f}")

# SVM one-hot
X_flat = X_onehot.reshape(len(X_onehot), -1).astype(np.float32)
svm_1h = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
    SVC(C=2.0, kernel="rbf", gamma="scale", probability=True, class_weight="balanced", random_state=0)
)
svm_1h.fit(X_flat[tr], y[tr])
q_tr = svm_1h.predict_proba(X_flat[tr])[:,1]
q_va = svm_1h.predict_proba(X_flat[va])[:,1]
q_te = svm_1h.predict_proba(X_flat[te])[:,1]

thr_1h = max([(t, f1_score(y[va], (q_va>=t).astype(int), zero_division=0))
              for t in np.linspace(0.1,0.9,33)], key=lambda x:x[1])[0]
m2_tr, y2_tr, cm2_tr = pack_metrics(y[tr], q_tr, "train", "SVM_onehot", thr=thr_1h)
m2_va, y2_va, cm2_va = pack_metrics(y[va], q_va, "val",   "SVM_onehot", thr=thr_1h)
m2_te, y2_te, cm2_te = pack_metrics(y[te], q_te, "test",  "SVM_onehot", thr=thr_1h)
J.log("svm_onehot", "ok", f"Val-opt thr={thr_1h:.2f}; test F1={m2_te['f1']:.3f}, AUC={m2_te['roc_auc']:.3f}, PR-AUC={m2_te['pr_auc']:.3f}")

# Cell 4 — QSVM (load kernels or rebuild tiny)

In [ ]:
K_trtr_p = RESULTS/"kernels/K_trtr.npy"; K_vatr_p = RESULTS/"kernels/K_vatr.npy"; K_tetr_p = RESULTS/"kernels/K_tetr.npy"
if K_trtr_p.exists() and K_vatr_p.exists() and K_tetr_p.exists():
    K_trtr = np.load(K_trtr_p); K_vatr = np.load(K_vatr_p); K_tetr = np.load(K_tetr_p)
    ytr_sel = y[tr][:K_trtr.shape[0]]
    J.log("qsvm_kernel", "ok", f"Loaded precomputed Gram matrices: {K_trtr.shape}, {K_vatr.shape}, {K_tetr.shape}")
else:
    J.log("qsvm_kernel", "warn", "Kernel cache not found; rebuilding small kernel (default.qubit).")
    D = 8
    pca = PCA(n_components=D, random_state=23)
    Xtr_p = pca.fit_transform(X_kmer[tr]); Xva_p = pca.transform(X_kmer[va]); Xte_p = pca.transform(X_kmer[te])
    to_ang = lambda X: (np.clip(X, -3, 3)/3.0)*np.pi
    Xtr_p, Xva_p, Xte_p = to_ang(Xtr_p), to_ang(Xva_p), to_ang(Xte_p)
    dev = qml.device("default.qubit", wires=D, shots=None)
    def U(x): qml.AngleEmbedding(x, wires=range(D), rotation="Y"); qml.broadcast(qml.CZ, wires=range(D), pattern="ring")
    @qml.qnode(dev)
    def kcirc(a,b): U(a); qml.adjoint(U)(b); return qml.expval(qml.Projector([0]*D, wires=range(D)))
    def gram(A,B):
        K = np.zeros((len(A), len(B)))
        for i in range(len(A)):
            for j in range(len(B)): K[i,j] = kcirc(A[i], B[j])
        return K
    K_trtr = gram(Xtr_p, Xtr_p); K_vatr = gram(Xva_p, Xtr_p); K_tetr = gram(Xte_p, Xtr_p)
    ytr_sel = y[tr]
    J.log("qsvm_kernel", "ok", f"Built Gram matrices: {K_trtr.shape}, {K_vatr.shape}, {K_tetr.shape}")

clf_q = SVC(C=5.0, kernel="precomputed", probability=True, class_weight="balanced", random_state=0)
clf_q.fit(K_trtr, ytr_sel)
r_tr = clf_q.predict_proba(K_trtr)[:,1]; r_va = clf_q.predict_proba(K_vatr)[:,1]; r_te = clf_q.predict_proba(K_tetr)[:,1]
thr_q = max([(t, f1_score(y[va], (r_va>=t).astype(int), zero_division=0)) for t in np.linspace(0.1,0.9,33)], key=lambda x:x[1])[0]
m3_tr, y3_tr, cm3_tr = pack_metrics(ytr_sel, r_tr, "train", "QSVM_kernel", thr=thr_q)
m3_va, y3_va, cm3_va = pack_metrics(y[va], r_va, "val",   "QSVM_kernel", thr=thr_q)
m3_te, y3_te, cm3_te = pack_metrics(y[te], r_te, "test",  "QSVM_kernel", thr=thr_q)
J.log("qsvm_fit", "ok", f"Val-opt thr={thr_q:.2f}; test F1={m3_te['f1']:.3f}, AUC={m3_te['roc_auc']:.3f}, PR-AUC={m3_te['pr_auc']:.3f}")

# Cell 5 — VQC eval (load weights; val-optimal thr)

In [ ]:
K_trtr_p = RESULTS/"kernels/K_trtr.npy"; K_vatr_p = RESULTS/"kernels/K_vatr.npy"; K_tetr_p = RESULTS/"kernels/K_tetr.npy"
if K_trtr_p.exists() and K_vatr_p.exists() and K_tetr_p.exists():
    K_trtr = np.load(K_trtr_p); K_vatr = np.load(K_vatr_p); K_tetr = np.load(K_tetr_p)
    ytr_sel = y[tr][:K_trtr.shape[0]]
    J.log("qsvm_kernel", "ok", f"Loaded precomputed Gram matrices: {K_trtr.shape}, {K_vatr.shape}, {K_tetr.shape}")
else:
    J.log("qsvm_kernel", "warn", "Kernel cache not found; rebuilding small kernel (default.qubit).")
    D = 8
    pca = PCA(n_components=D, random_state=23)
    Xtr_p = pca.fit_transform(X_kmer[tr]); Xva_p = pca.transform(X_kmer[va]); Xte_p = pca.transform(X_kmer[te])
    to_ang = lambda X: (np.clip(X, -3, 3)/3.0)*np.pi
    Xtr_p, Xva_p, Xte_p = to_ang(Xtr_p), to_ang(Xva_p), to_ang(Xte_p)
    dev = qml.device("default.qubit", wires=D, shots=None)
    def U(x): qml.AngleEmbedding(x, wires=range(D), rotation="Y"); qml.broadcast(qml.CZ, wires=range(D), pattern="ring")
    @qml.qnode(dev)
    def kcirc(a,b): U(a); qml.adjoint(U)(b); return qml.expval(qml.Projector([0]*D, wires=range(D)))
    def gram(A,B):
        K = np.zeros((len(A), len(B)))
        for i in range(len(A)):
            for j in range(len(B)): K[i,j] = kcirc(A[i], B[j])
        return K
    K_trtr = gram(Xtr_p, Xtr_p); K_vatr = gram(Xva_p, Xtr_p); K_tetr = gram(Xte_p, Xtr_p)
    ytr_sel = y[tr]
    J.log("qsvm_kernel", "ok", f"Built Gram matrices: {K_trtr.shape}, {K_vatr.shape}, {K_tetr.shape}")

clf_q = SVC(C=5.0, kernel="precomputed", probability=True, class_weight="balanced", random_state=0)
clf_q.fit(K_trtr, ytr_sel)
r_tr = clf_q.predict_proba(K_trtr)[:,1]; r_va = clf_q.predict_proba(K_vatr)[:,1]; r_te = clf_q.predict_proba(K_tetr)[:,1]
thr_q = max([(t, f1_score(y[va], (r_va>=t).astype(int), zero_division=0)) for t in np.linspace(0.1,0.9,33)], key=lambda x:x[1])[0]
m3_tr, y3_tr, cm3_tr = pack_metrics(ytr_sel, r_tr, "train", "QSVM_kernel", thr=thr_q)
m3_va, y3_va, cm3_va = pack_metrics(y[va], r_va, "val",   "QSVM_kernel", thr=thr_q)
m3_te, y3_te, cm3_te = pack_metrics(y[te], r_te, "test",  "QSVM_kernel", thr=thr_q)
J.log("qsvm_fit", "ok", f"Val-opt thr={thr_q:.2f}; test F1={m3_te['f1']:.3f}, AUC={m3_te['roc_auc']:.3f}, PR-AUC={m3_te['pr_auc']:.3f}")

# Cell 6 — save combined + ROC/CM caches

In [ ]:
# Combine metrics
df = pd.DataFrame([m1_tr,m1_va,m1_te, m2_tr,m2_va,m2_te, m3_tr,m3_va,m3_te, m4_tr,m4_va,m4_te])
df.to_csv(RESULTS/"metrics/combined.csv", index=False)

# Save ROC probability caches (test)
ROC_DIR = RESULTS/"roc_cache"; ROC_DIR.mkdir(parents=True, exist_ok=True)
np.save(ROC_DIR/"y_test.npy", y[te])
np.save(ROC_DIR/"probs_svm_kmer.npy", p_te)
np.save(ROC_DIR/"probs_svm_onehot.npy", q_te)
np.save(ROC_DIR/"probs_qsvm_kernel.npy", r_te)
np.save(ROC_DIR/"probs_vqc.npy", s_te)

# Save CM caches (raw predictions at model-specific best thr)
CM_DIR = RESULTS/"cm_cache"; CM_DIR.mkdir(parents=True, exist_ok=True)
(Path(CM_DIR/"y_true.json")).write_text(json.dumps(y[te].astype(int).tolist()))
for name, arr, thr in [
    ("svm_kmer",    p_te, m1_te["thr"]),
    ("svm_onehot",  q_te, m2_te["thr"]),
    ("qsvm_kernel", r_te, m3_te["thr"]),
    ("vqc",         s_te, m4_te["thr"])
]:
    (Path(CM_DIR)/f"y_pred_{name}.json").write_text(json.dumps((arr>=thr).astype(int).tolist()))

# Also persist per-model classification reports (test) for documentation
reports = {}
for name, ytrue, prob, thr in [
    ("SVM_kmer", y[te], p_te, m1_te["thr"]),
    ("SVM_onehot", y[te], q_te, m2_te["thr"]),
    ("QSVM_kernel", y[te], r_te, m3_te["thr"]),
    ("VQC", y[te], s_te, m4_te["thr"])
]:
    yhat = (prob>=thr).astype(int)
    reports[name] = classification_report(ytrue, yhat, output_dict=True, zero_division=0)
with open(RESULTS/"metrics/classification_reports_test.json", "w", encoding="utf-8") as f:
    json.dump(reports, f, indent=2)

# Nice quick view
df[df.split=="test"].sort_values("f1", ascending=False)

# Cell 7 — ROC plot

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))

def plot_roc(y_true, y_prob, label):
    try:
        auc = roc_auc_score(y_true, y_prob)
        fpr, tpr, _ = roc_curve(y_true, y_prob)
        ax.plot(fpr, tpr, label=f"{label} (AUC={auc:.3f})")
    except Exception as e:
        print(f"ROC for {label} skipped: {e}")

plot_roc(y[te], p_te, "SVM_kmer")
plot_roc(y[te], q_te, "SVM_onehot")
plot_roc(y[te], r_te, "QSVM_kernel")
plot_roc(y[te], s_te, "VQC")
ax.plot([0,1], [0,1], "--", lw=1); ax.set_xlabel("False Positive Rate"); ax.set_ylabel("True Positive Rate"); ax.set_title("ROC (test)"); ax.legend()
plt.tight_layout(); plt.show()

# Cell 8 — confusion matrices

In [ ]:
def _show_cm(name, cm):
    fig, ax = plt.subplots(figsize=(3.5,3.2))
    im = ax.imshow(cm, interpolation="nearest")
    ax.set_title(f"Confusion Matrix — {name} (test)")
    ax.set_xlabel("Predicted"); ax.set_ylabel("True")
    for (i,j), v in np.ndenumerate(cm):
        ax.text(j, i, int(v), ha="center", va="center", color="white" if im.norm(v) > 0.5 else "black")
    plt.tight_layout(); plt.show()

cms = [("SVM_kmer", cm1_te), ("SVM_onehot", cm2_te), ("QSVM_kernel", cm3_te), ("VQC", cm4_te)]
for name, cm in cms:
    _show_cm(name, cm)

# Save raw & normalized CM CSVs (test) for reproducibility
def _save_cm_csv(cm, out_csv, normalized=False):
    arr = cm.astype(np.float64)
    if normalized:
        rs = arr.sum(axis=1, keepdims=True)
        arr = np.divide(arr, np.where(rs==0, 1, rs))
    df_cm = pd.DataFrame(arr, index=["true_0","true_1"], columns=["pred_0","pred_1"])
    df_cm.to_csv(out_csv, index=True)

for name, cm in cms:
    _save_cm_csv(cm, RESULTS/f"metrics/{name.lower()}_cm_test.csv", normalized=False)
    _save_cm_csv(cm, RESULTS/f"metrics/{name.lower()}_cm_test_norm.csv", normalized=True)

# Cell 9 — McNemar tests

In [ ]:
from math import fabs, exp

def mcnemar(y_true, yhat_a, yhat_b):
    n01 = int(((yhat_a == y_true) & (yhat_b != y_true)).sum())
    n10 = int(((yhat_a != y_true) & (yhat_b == y_true)).sum())
    # continuity-corrected chi^2
    chi2 = (fabs(n01 - n10) - 1)**2 / (n01 + n10 + 1e-12)
    p = exp(-chi2/2)  # simple approx; note: for exact p use binomial test
    return {"n01":n01, "n10":n10, "chi2":chi2, "p_approx":p}

stats = {
    "QSVM_vs_SVMkmer": mcnemar(y[te], (p_te>=m1_te["thr"]).astype(int), (r_te>=m3_te["thr"]).astype(int)),
    "VQC_vs_SVMkmer":  mcnemar(y[te], (p_te>=m1_te["thr"]).astype(int), (s_te>=m4_te["thr"]).astype(int)),
    "SVM1hot_vs_SVMkmer": mcnemar(y[te], (p_te>=m1_te["thr"]).astype(int), (q_te>=m2_te["thr"]).astype(int)),
}
with open(RESULTS/"stats/mcnemar.json","w") as f: json.dump(stats, f, indent=2)
stats

# --- Save run journal (what worked, what didn't, why) ---
ts = time.strftime("%Y%m%d_%H%M%S")
J.log("summary", "ok",
      f"Best test F1: {df[df.split=='test'].sort_values('f1', ascending=False).iloc[0]['model']}")
J.save(RESULTS/"logs"/f"benchmark_analysis_{ts}")

# Also write a quick textual roll-up of warnings/failures
issues = [f"- [{e['step']}] {e['message']}" for e in J.events if e["status"] in ("warn","fail")]
rollup = "No warnings or failures." if not issues else "Issues observed:\n" + "\n".join(issues)
(Path(RESULTS/"logs"/f"benchmark_analysis_{ts}_summary.txt")).write_text(rollup, encoding="utf-8")